# Dask Array

Материалы:
* Макрушин С.В. Лекция 11: Dask
* https://docs.dask.org/en/latest/array.html
* JESSE C. DANIEL. Data Science with Python and Dask. 

## Задачи для совместного разбора

1. Создайте массив размерностью 1000 на 300000, заполненный числами из стандартного нормального распределения. Исследуйте основные характеристики полученного массива.

2. Посчитайте сумму квадратов элементов массива, созданного в задаче 1. Создайте массив `np.array` такого же размера и сравните скорость решения задачи с использование `da.array` и `np.array`

3. Визуализируйте граф вычислений для задачи 12.

## Лабораторная работа 11

In [1]:
import dask.array as da
import h5py
import numpy as np

In [2]:
!pip install --upgrade dask

In [3]:
!pip install --user --upgrade dask

In [4]:
import dask
dask.__version__

'2021.11.2'

1. Считайте датасет `recipe` из файла `minutes_n_ingredients_full.hdf5` в виде `dask.array`. Укажите аргумент `chunks=(100_000, 3)` при создании массива. Выведите на экран основную информацию о массиве.

In [6]:
recipes = da.from_array(
    h5py.File('data/minutes_n_ingredients_full.hdf5')['recipe'],
    chunks = (100_000, 3)
)
recipes

dask.array<array, shape=(2231637, 3), dtype=int64, chunksize=(100000, 3), chunktype=numpy.ndarray>

2. Вычислите среднее значение по каждому столбцу, кроме первого. 

In [7]:
mean = recipes[:, 1:].mean(axis=0).compute()
mean

array([1004.20805176,    5.4198008 ])

3. Исследуйте, как влияет значение аргумента `chunks` при создании `dask.array` на скорость выполнения операции поиска среднего. 

In [8]:
def find_mean(arr):
    return arr[:, 1:].mean(axis=0).compute()

In [9]:
%timeit find_mean(recipes)

49.7 ms ± 815 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
_recipes = recipes.rechunk((10_000, 100))
%timeit find_mean(_recipes)

114 ms ± 7.15 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
_recipes = recipes.rechunk((100_000, 10_000))
%timeit find_mean(_recipes)

49.1 ms ± 990 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
_recipes = recipes.rechunk((500, 500))
%timeit find_mean(_recipes)

1.58 s ± 13.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


4. Выберите рецепты, время выполнения которых меньше медианного значения

In [13]:
recipes[recipes[:, 1] < da.median(recipes[:, 1], axis=0)].compute()

array([[1089012,      23,       5],
       [1428572,       0,       5],
       [1400250,      24,       1],
       ...,
       [1029131,      19,       4],
       [1700703,       1,       1],
       [ 713836,       0,       9]], dtype=int64)

5. Посчитайте количество каждого из возможных значений кол-ва ингредиентов

In [14]:
unique, counts = da.unique(recipes[:, 2], return_counts=True)

In [15]:
unique = da.from_array([unique]).T
counts = da.from_array([counts]).T
da.hstack([unique, counts]).compute()

array([[     1, 222071],
       [     2, 224158],
       [     3, 229388],
       [     4, 234948],
       [     5, 240720],
       [     6, 244360],
       [     7, 247181],
       [     8, 246747],
       [     9, 246816],
       [    10,  22430],
       [    11,  19094],
       [    12,  15165],
       [    13,  11640],
       [    14,   8284],
       [    15,   6014],
       [    16,   4145],
       [    17,   2793],
       [    18,   1913],
       [    19,   1279],
       [    20,    852],
       [    21,    529],
       [    22,    346],
       [    23,    244],
       [    24,    178],
       [    25,    107],
       [    26,     68],
       [    27,     55],
       [    28,     33],
       [    29,     22],
       [    30,     20],
       [    31,     13],
       [    32,      5],
       [    33,      4],
       [    34,      3],
       [    35,      4],
       [    36,      1],
       [    37,      2],
       [    38,      1],
       [    39,      1],
       [    40,      2],


6. Найдите максимальную продолжительность рецепта. Ограничьте максимальную продолжительность рецептов сверху значением, равному 75% квантилю.

In [16]:
max_minutes = recipes[:, 1].max()
max_minutes.compute()

2147483647

In [17]:
recipes_minutes_75 = recipes.copy()
quantile = np.quantile(recipes_minutes_75[:, 1].compute(), 0.75)
recipes_minutes_75[:, 1] = da.where(
    recipes_minutes_75[:, 1] <= quantile,
    recipes_minutes_75[:, 1],
    quantile
)
recipes_minutes_75 = recipes_minutes_75.compute()
recipes_minutes_75

array([[ 683970,      33,       9],
       [1089012,      23,       5],
       [1428572,       0,       5],
       ...,
       [1910650,      49,       2],
       [ 713836,       0,       9],
       [ 660699,      49,       8]], dtype=int64)

In [18]:
recipes_minutes_75[:, 1].max()

49

7. Создайте массив `dask.array` из 2 чисел, содержащих ваши предпочтения относительно времени выполнения рецепта и кол-ва ингредиентов. Найдите наиболее похожий (в смысле $L_1$) рецепт из имеющихся в датасете.

In [19]:
favorite = da.array([10, 4])

In [20]:
l1_distance = da.linalg.norm(recipes[:, 1:] - favorite, ord = 1, axis = 1)

In [21]:
most_similar = recipes[l1_distance.argmin()].compute()
most_similar

array([144542,     10,      4], dtype=int64)

8. Работая с исходным файлом в формате `hdf5`, реализуйте алгоритм подсчета среднего значения в блочной форме и вычислите с его помощью среднее значение второго столбца в массиве.

Блочный алгоритм вычислений состоит из двух частей:
1. Загрузка фрагмента за фрагментом данных по `blocksize` элементов и проведение вычислений на этим фрагментом
2. Агрегация результатов вычислений на различных фрагментах для получения результата на уровне всего набора данных

Важно: при работе с `h5py` в память загружаются не все элементы, а только те, которые запрашиваются в данный момент

In [22]:
def block_algorithm(h5_file, dataset = 'recipe', col = 1, blocksize = 100_000):
    with h5py.File(h5_file) as f:
        dataset = f[dataset]
        sizes = []
        sums = []
        for i in range(0, dataset.shape[0], blocksize):
            arr = dataset[i:i + blocksize, col]
            sums.append(arr.sum())
            sizes.append(arr.size)
        return sum(sums) / sum(sizes)

In [23]:
block_algorithm('data/minutes_n_ingredients_full.hdf5', blocksize = 100_000)

1004.2080517575215